<a href="https://colab.research.google.com/github/ekunnii/chatbot_feeder/blob/master/notebooks/ParlAI_self_feeding_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount mila gdrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/ParlAI")

In [4]:
# !git clone https://github.com/facebookresearch/ParlAI.git
!python setup.py develop

running develop
running egg_info
writing parlai.egg-info/PKG-INFO
writing dependency_links to parlai.egg-info/dependency_links.txt
writing entry points to parlai.egg-info/entry_points.txt
writing requirements to parlai.egg-info/requires.txt
writing top-level names to parlai.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'parlai.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.6/dist-packages/parlai.egg-link (link to .)
Adding parlai 0.1.20200526 to easy-install.pth file

Installed /content/drive/My Drive/ParlAI
Processing dependencies for parlai==0.1.20200526
Searching for websocket-server==0.4
Reading https://pypi.org/simple/websocket-server/
Best match: websocket-server 0.4
Processing websocket_server-0.4.tar.gz
Writing /tmp/easy_install-scqn4r2o/websocket_server-0.4/setup.cfg
Running websocket_server-0.4/setup.py -q bdist_egg --dist-dir /tmp/easy_install-scqn4r2o/websocket_server-0.4/egg-dist-tmp-5hne8gj0
zip_safe flag not

In [0]:
# !python projects/self_feeding/download_data.py

In [0]:
!python examples/train_model.py -t self_feeding:dialog --model projects.self_feeding.self_feeding_agent:SelfFeedingAgent --model-file tmp/mymodel1 -bs 128

In [0]:
!python examples/train_model.py -t self_feeding:all --model-file tmp/mymodel4 -ltim 5 -vtim 10 -vp 50 -m projects.self_feeding.self_feeding_agent:SelfFeedingAgent -cands batch --eval-candidates inline -histsz 2 --embedding-type fasttext_cc --embedding-size 300 --dict-maxtokens 250000 --num-epochs 500 --optimizer adamax --embeddings-scale false -bs 128 --relu-dropout 0 --attention-dropout 0 --n-heads 6 --n-layers 6 -lr 0.0025 --ffn-size 32 --lr-scheduler invsqrt --warmup-updates 500 --dia-train train_hh131k_hb60k.txt -vmt dia_acc -vmm max

[ building dictionary first... ]
[ dictionary already built .]
[ Using CUDA ]
Dictionary: loading dictionary from tmp/mymodel4.dict
[ num words =  25141 ]
tcmalloc: large alloc 2400002048 bytes == 0x18aa4000 @  0x7f8cc898ab6b 0x7f8cc89aa379 0x7f8c7042c04e 0x7f8c7042df4a 0x7f8ca931b0c4 0x7f8cb75685d9 0x551365 0x5a9cbc 0x50a5c3 0x50bfb4 0x507d64 0x509042 0x5a4b71 0x5a4ec8 0x4e009e 0x50a5f1 0x50bfb4 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509042
^C


# Polyencoder

## Interactive example

In [5]:
!pip install 'git+https://github.com/rsennrich/subword-nmt.git#egg=subword-nmt'

  Cloning https://github.com/rsennrich/subword-nmt.git to /tmp/pip-install-t_w3t626/subword-nmt
  Running command git clone -q https://github.com/rsennrich/subword-nmt.git /tmp/pip-install-t_w3t626/subword-nmt
  Created wheel for subword-nmt: filename=subword_nmt-0.3.7-cp36-none-any.whl size=128391 sha256=ca60a06216d648a8918ea7719c5696e3470f635e06277b293dd52b682d0dcd48
  Stored in directory: /tmp/pip-ephem-wheel-cache-p4byjqmq/wheels/05/2b/3a/73cd213b79c18512c76a9ea40329a6819fa9276e232679d834
Successfully built subword-nmt


In [6]:
!python examples/interactive.py -m transformer/polyencoder \
    -mf zoo:pretrained_transformers/model_poly/model \
    --encode-candidate-vecs true \
    --eval-candidates fixed  \
    --fixed-candidates-path data/models/pretrained_transformers/convai_trainset_cands.txt

Traceback (most recent call last):
  File "examples/interactive.py", line 12, in <module>
    from parlai.scripts.interactive import Interactive
  File "/content/drive/My Drive/ParlAI/parlai/scripts/interactive.py", line 21, in <module>
    from parlai.core.params import ParlaiParser
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 764, in get_code
  File "<frozen importlib._bootstrap_external>", line 833, in get_data
KeyboardInterrupt


## bi-encoder

In [6]:
!python -u examples/train_model.py \
    --init-model zoo:pretrained_transformers/bi_model_huge_reddit/model \
    --batchsize 16 -t self_feeding:dialog \
    --model transformer/biencoder --eval-batchsize 6 \
    --warmup_updates 100 --lr-scheduler-patience 0 \
    --lr-scheduler-decay 0.4 -lr 5e-05 --data-parallel True \
    --history-size 20 --label-truncate 72 --text-truncate 360 \
    --num-epochs 10.0 --max_train_time 200000 -veps 0.5 -vme 8000 \
    --validation-metric accuracy --validation-metric-mode max \
    --save-after-valid True --log_every_n_secs 20 --candidates batch \
    --dict-tokenizer bpe --dict-lower True --optimizer adamax \
    --output-scaling 0.06 \
     --variant xlm --reduction-type mean --share-encoders False \
     --learn-positional-embeddings True --n-layers 12 --n-heads 12 \
     --ffn-size 3072 --attention-dropout 0.1 --relu-dropout 0.0 --dropout 0.1 \
     --n-positions 1024 --embedding-size 768 --activation gelu \
     --embeddings-scale False --n-segments 2 --learn-embeddings True \
     --share-word-embeddings False --dict-endtoken __start__ --fp16 True --force-fp16-tokens True\
     --model-file tmp/bi-encoder-selffeeding

Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 88, in _path_is_mode_type
  File "<frozen importlib._bootstrap_external>", line 82, in _path_stat
FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.6/dist-packages/torch/nn/utils/__init__.cpython-36m-x86_64-linux-gnu.so'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "examples/train_model.py", line 12, in <module>
    from parlai.scripts.train_model import TrainModel
  File "/content/drive/My Drive/ParlAI/parlai/scripts/train_model.py", line 33, in <module>
    from parlai.core.metrics import Metric
  File "/content/drive/My Drive/ParlAI/parlai/core/metrics.py", line 20, in <module>
    import torch
  File "/usr/local/lib/python3.6/dist-packages/torch/__init__.py", line 331, in <module>
    from .functional import *
  File "/usr/local/lib/python3.6/dist-packages/torch/functional.py", line 2, in <module>
 

## Polyencoder

### Polyencoder and only train_hh.txt

In [12]:
!python -u examples/train_model.py \
  --init-model zoo:pretrained_transformers/poly_model_huge_reddit/model \
  -t self_feeding:dialog \
  --model transformer/polyencoder --batchsize 16 --eval-batchsize 10 \
  --warmup_updates 100 --lr-scheduler-patience 0 --lr-scheduler-decay 0.4 \
  -lr 5e-05 --data-parallel True --history-size 20 --label-truncate 72 \
  --text-truncate 360 --num-epochs 8.0 --max_train_time 200000 -veps 0.05 \
  -vme 8000 --validation-metric accuracy --validation-metric-mode max \
  --save-after-valid True --log_every_n_secs 20 --candidates batch --fp16 True \
  --dict-tokenizer bpe --dict-lower True --optimizer adamax --output-scaling 0.06 \
  --variant xlm --reduction-type mean --share-encoders False \
  --learn-positional-embeddings True --n-layers 12 --n-heads 12 --ffn-size 3072 \
  --attention-dropout 0.1 --relu-dropout 0.0 --dropout 0.1 --n-positions 1024 \
  --embedding-size 768 --activation gelu --embeddings-scale False --n-segments 2 \
  --learn-embeddings True --polyencoder-type codes --poly-n-codes 64 \
  --poly-attention-type basic --dict-endtoken __start__ \
  --model-file tmp3/polyencoder --dia-train train_fb.txt

[ building dictionary first... ]
[creating task(s): self_feeding:dialog]
[ Loading Self-Feeding text data:/content/drive/My Drive/ParlAI/data/self_feeding/train_fb.txt]
Building dictionary: 100% 61.3k/61.3k [00:02<00:00, 29.0kex/s]
Dictionary: saving bpe codecs to tmp3/polyencoder.dict.codecs
no pair has frequency >= 2. Stopping
Dictionary: saving dictionary to tmp3/polyencoder.dict
[ dictionary built with 13680 tokens in 0s ]
[ no model with opt yet at: tmp3/polyencoder(.opt) ]

***************************************************************************
[ WARNING ] : your model is being loaded with opts that do not exist in the model you are initializing the weights with: dynamic_batching: None,evaltask: None,eval_batchsize: 10,display_examples: False,num_epochs: 8.0,max_train_time: 200000.0,validation_every_n_secs: -1,save_every_n_secs: -1,save_after_valid: True,validation_every_n_epochs: 0.05,validation_max_exs: 8000,short_final_eval: False,validation_patience: 10,validation_metric:

### Polyencoder with dialog and feedback data

In [0]:
! cd data/self_feeding && cat train_hh.txt > train_hh_fb.txt && cat train_fb.txt >> train_hh_fb.txt

In [0]:
!python -u examples/train_model.py \
  --init-model zoo:pretrained_transformers/poly_model_huge_reddit/model \
  -t self_feeding:dialog \
  --model transformer/polyencoder --batchsize 16 --eval-batchsize 10 \
  --warmup_updates 100 --lr-scheduler-patience 0 --lr-scheduler-decay 0.4 \
  -lr 5e-05 --data-parallel True --history-size 20 --label-truncate 72 \
  --text-truncate 360 --num-epochs 8.0 --max_train_time 200000 -veps 0.5 \
  -vme 8000 --validation-metric accuracy --validation-metric-mode max \
  --save-after-valid True --log_every_n_secs 20 --candidates batch --fp16 True \
  --dict-tokenizer bpe --dict-lower True --optimizer adamax --output-scaling 0.06 \
  --variant xlm --reduction-type mean --share-encoders False \
  --learn-positional-embeddings True --n-layers 12 --n-heads 12 --ffn-size 3072 \
  --attention-dropout 0.1 --relu-dropout 0.0 --dropout 0.1 --n-positions 1024 \
  --embedding-size 768 --activation gelu --embeddings-scale False --n-segments 2 \
  --learn-embeddings True --polyencoder-type codes --poly-n-codes 64 \
  --poly-attention-type basic --dict-endtoken __start__ \
  --model-file tmp/polyencoder_hh_fb --dia-train train_hh_fb.txt

Streaming output truncated to the last 5000 lines.
   .9445   
    rank  \
   1.146   
    total_train_updates  \
                  49748   
     tpb  \
   214.5   
    train_accuracy  \
             .9006   
    updates  
         39

[ time:28052.0s total_exs:796576 epochs:4.13 time_left:26259.0s ]
    clip  \
       1   
    exs  \
    608   
    gnorm  \
    23.21   
    gpu_mem  \
      1.097   
         lr  \
   1.28e-06   
    mean_loss  \
        .2302   
     mrr  \
   .9605   
    rank  \
   1.104   
    total_train_updates  \
                  49786   
     tpb  \
   213.7   
    train_accuracy  \
             .9276   
    updates  
         38

[ time:28072.0s total_exs:797200 epochs:4.14 time_left:26236.0s ]
    clip  \
       1   
    exs  \
    624   
    gnorm  \
    24.53   
    gpu_mem  \
      1.097   
         lr  \
   1.28e-06   
    mean_loss  \
        .2924   
     mrr  \
   .9447   
    rank  \
   1.143   
    total_train_updates  \
                  49825   
 